In [1]:
import pandas as pd
final = pd.read_csv(r'final.csv', encoding = 'CP949')
final.head(5)

,시도,시군구,행정번호,년도,우울증환자수,평균연령,총인구수,스트레스인지율,고용률,코로나확진자수,1인당 정신건강예산(원),공공시설개수,우울증지수
0,서울특별시,서울종로구,1111000000,2018,14837,44.2,153065,34.2,59.6,0,5326,44,9.693268
1,서울특별시,서울중구,1114000000,2018,4352,44.6,125725,28.3,59.6,0,5326,42,3.461523
2,서울특별시,서울용산구,1117000000,2018,2370,43.6,228999,20.0,59.6,0,5326,53,1.034939
3,서울특별시,서울성동구,1120000000,2018,3657,42.0,308221,31.3,59.6,0,5326,50,1.186486
4,서울특별시,서울광진구,1121500000,2018,6917,41.4,355559,29.4,59.6,0,5326,47,1.945387


In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [10]:
X = final[['우울증환자수', '평균연령', '총인구수', '스트레스인지율', '고용률', '코로나확진자수', '1인당 정신건강예산(원)', '공공시설개수']]
y = final['우울증지수']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# SVR 모델 정의
svr = SVR(kernel = 'rbf')

# 그리드 서치를 위한 매개변수 그리드 정의
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [0.01, 0.1, 1, 10]}

# GridSearchCV 객체 생성
grid_search = GridSearchCV(svr, param_grid, cv=5)

# 그리드 서치 수행
grid_search.fit(X_train, y_train)

# 최적의 매개변수 출력
print("최적의 매개변수:", grid_search.best_params_)

# 최적의 모델 출력
best_model = grid_search.best_estimator_

최적의 매개변수: {'C': 10, 'gamma': 0.01}


In [14]:
# 모델 파이프라인 생성 
svr_model = make_pipeline(
    RobustScaler(),
    SVR(kernel='rbf', C = 10, gamma = 0.1)
)

# 모델 성능 검정
rf_cv_scores_r2 = cross_val_score(svr_model, X, y, cv=5, scoring='r2')
rf_cv_scores_mse = cross_val_score(svr_model, X, y, cv=5, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-rf_cv_scores_mse)

print('cv=' , 5, '/ 평균 r²:', round(rf_cv_scores_r2.mean(),4), '/ 표준편차 r²:', round(rf_cv_scores_r2.std(),4), '/ rmse :', round(cv_rmse.mean(),4))

cv= 5 / 평균 r²: 0.708 / 표준편차 r²: 0.3952 / rmse : 0.5518


그리드 서치 진행했으나 다른 하이퍼파라미터 조합이 교차검증 했을 때 평균 r² 점수가 더 높게 나옴